In [11]:
import openpyxl

def update_excel_file(tanggal, hari, jam, kelompok, operator, NIP):
    # Load the workbook and select the active worksheet
    workbook = openpyxl.load_workbook('QC Seiscomp.xlsx')
    sheet = workbook.active

    # Fill cell G2 with the specified value
    sheet['G2'] = ': ' + tanggal
    sheet['G3'] = ': ' + hari
    sheet['G4'] = f': {jam} - selesai'
    sheet['G5'] = f': {kelompok}'
    sheet['M18'] = tanggal
    sheet['M24'] = operator
    NIP = sheet['M25'] = 'NIP. ' + NIP

    # Save the workbook
    workbook.save('QC Seiscomp.xlsx')

# Example usage
update_excel_file('20 Desember 2020', 'Senin', '14:00', 'Kel. 1', 'Fajar', '199411052014111001')

In [7]:
import pandas as pd
from io import StringIO
from openpyxl.utils.dataframe import dataframe_to_rows
# Sample CSV string
csv_string = "apple,banana,cherry,orange\n1,2,3,4\n5,6,7,8"

# Create a DataFrame from the CSV string
df = pd.read_csv(StringIO(csv_string))
rows = dataframe_to_rows(df, index=False, header=False)

# print df rows size
print(len(df))
df['prev'] = 'prev'
df['no'] = range(1, len(df)+1)
print(df.shape[1])

for r_idx, row in enumerate(rows, 1):
    print(r_idx, row)

2
6
1 [1, 2, 3, 4, 'prev', 1]
2 [5, 6, 7, 8, 'prev', 2]


In [2]:
import subprocess
import os

def convert_xlsx_to_pdf(filename):
  """
  Converts an XLSX file to PDF using the LibreOffice command-line interface.

  Args:
    filename: Path to the input XLSX file.

  Returns:
    True if the conversion was successful, False otherwise.
  """
  try:
    command = ["soffice", "--headless", "--convert-to", "pdf:\"impress_pdf_Export\"", filename]
    subprocess.run(command, check=True) 
    return True
  except subprocess.CalledProcessError as e:
    print(f"Error converting {filename} to PDF: {e}")
    return False

# Example usage
filename = os.path.join(os.getcwd(), 'QC.xlsx')
print(filename)
# convert_xlsx_to_pdf(filename)

d:\Projects\QC\ebast\qc\static\qc\QC.xlsx


In [ ]:
import requests
import pandas as pd

def clean_fm_data(data, start_datetime='2025-03-12 00:00:00', end_datetime='2025-03-13 00:00:00'):
    text = data.decode('utf-8')

    lines = text.split('\n')
    processed_lines = []
    for i, line in enumerate(lines):
        line = '|'.join(part.strip() for part in line.split('|'))
        processed_lines.append(line)

    df = pd.DataFrame([x.split('|') for x in processed_lines[1:]], columns=processed_lines[0].split('|'))
    print(df)
    df['Datetime (UTC)'] = pd.to_datetime(df['Datetime (UTC)'], format='%Y-%m-%d %H:%M:%S')

    def select_data_by_datetime_range(df, start_datetime, end_datetime):
        mask = (df['Datetime (UTC)'] >= start_datetime) & (df['Datetime (UTC)'] <= end_datetime)
        return df.loc[mask]

    df_selected = select_data_by_datetime_range(df, start_datetime, end_datetime)

    # sort the df_selected by 'Datetime (UTC)'
    df_selected = df_selected.sort_values(by='Datetime (UTC)')

    # divide the 'Datetime (UTC)' column into 'Date' and 'OT (UTC)' columns, and put them in the first two columns, remove the 'Datetime (UTC)' column
    df_selected['Date'] = df_selected['Datetime (UTC)'].dt.date
    df_selected['OT (UTC)'] = df_selected['Datetime (UTC)'].dt.time
    df_selected = df_selected[['Date', 'OT (UTC)'] + [col for col in df_selected.columns if col != 'Datetime (UTC)']]
    df_selected = df_selected.reset_index(drop=True)

    # sort the columns to be 'Date', 'OT (UTC)', 'Lat', 'Long', 'Mag', 'D(Km)', 'Phase', 'RMS', 'Az.Gap', 'Region', but first turn the respective column names into the desired ones
    df_selected = df_selected.rename(columns={'D': 'D(Km)', 'Type M': 'TypeMag'})
    df_selected = df_selected[['Date', 'OT (UTC)', 'Lat', 'Long', 'Mag', 'TypeMag', 'D(Km)', 'S1', 'D1', 'R1', 'S2', 'D2', 'R2', 'Fit(%)', 'CLVD (%)']]
    df_selected = df_selected.reset_index(drop=True)

    # Check for duplicate columns
    df_selected = df_selected.loc[:, ~df_selected.columns.duplicated()]
    
    return df_selected

def fetch_data(start_datetime='2025-03-12 00:00:00', end_datetime='2025-03-13 00:00:00'):
    url = "http://202.90.198.41/qc_focal.txt"
    response = requests.get(url)

    if response.status_code == 200:
        data = clean_fm_data(response.content, start_datetime, end_datetime)
        csv_data = data.to_csv(index=False)
        table_data = data.insert(0, 'No', range(1, len(data) + 1))
        table_data = data.to_dict(orient='records')
        return table_data
    #     return JsonResponse({'csv': csv_data, 'table_data': table_data})
    # else:
    #     return JsonResponse({'error': 'Failed to fetch data'}, status=500)
    
fetch_data()

         EventID       Datetime (UTC)  Creation Time (UTC)       Mode  \
0    gfz2025ehnv  2025-03-03 01:04:52  2025-03-03 02:28:48     manual   
1    gfz2025ehmf  2025-03-03 00:15:30  2025-03-03 00:20:49  automatic   
2    gfz2025ehep  2025-03-02 20:24:30  2025-03-02 20:33:28  automatic   
3    gfz2025ehdv  2025-03-02 20:01:32  2025-03-02 20:09:38  automatic   
4    gfz2025ehci  2025-03-02 19:16:48  2025-03-02 19:22:08  automatic   
..           ...                  ...                  ...        ...   
175  bmg2025acjo  2025-01-02 07:06:40  2025-01-02 09:08:21  automatic   
176  bmg2025acgm  2025-01-02 05:32:21  2025-01-02 05:44:24     manual   
177  bmg2025abbi  2025-01-01 13:48:18  2025-01-01 14:34:28  automatic   
178                              None                 None       None   
179                              None                 None       None   

        Status   Mag Type M       Lat      Long     D  ...  Tplg  Tazi   Nval  \
0    confirmed   4.0     Mw    0.9312  100

[{'No': 1,
  'Date': datetime.date(2025, 3, 12),
  'OT (UTC)': datetime.time(5, 42, 6),
  'Lat': '6.6609',
  'Long': '92.0575',
  'Mag': '5.9',
  'TypeMag': 'Mw',
  'D(Km)': '10',
  'S1': '175',
  'D1': '59',
  'R1': '112',
  'S2': '317',
  'D2': '37',
  'R2': '58',
  'Fit(%)': '84.90',
  'CLVD (%)': '30.57'},
 {'No': 2,
  'Date': datetime.date(2025, 3, 12),
  'OT (UTC)': datetime.time(12, 51, 31),
  'Lat': '-7.5164',
  'Long': '129.2063',
  'Mag': '5.0',
  'TypeMag': 'Mw',
  'D(Km)': '124',
  'S1': '85',
  'D1': '51',
  'R1': '67',
  'S2': '300',
  'D2': '45',
  'R2': '116',
  'Fit(%)': '88.16',
  'CLVD (%)': '38.25'},
 {'No': 3,
  'Date': datetime.date(2025, 3, 12),
  'OT (UTC)': datetime.time(15, 9, 50),
  'Lat': '8.2152',
  'Long': '127.5588',
  'Mag': '5.6',
  'TypeMag': 'Mw',
  'D(Km)': '54',
  'S1': '106',
  'D1': '40',
  'R1': '-156',
  'S2': '358',
  'D2': '75',
  'R2': '-52',
  'Fit(%)': '81.91',
  'CLVD (%)': '46.69'}]